In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import mlflow
# mlflow ui --backend-store-uri sqlite:///mlflow.db
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-expirement")

<Experiment: artifact_location='/workspaces/mlops/02-experiment-tracking/mlruns/1', creation_time=1717557366335, experiment_id='1', last_update_time=1717557366335, lifecycle_stage='active', name='nyc-taxi-expirement', tags={}>

In [5]:
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [6]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [7]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.000,0.970,1.000,N,161,141,2,9.300,1.000,0.500,0.000,0.000,1.000,14.300,2.500,0.000
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.000,1.100,1.000,N,43,237,1,7.900,1.000,0.500,4.000,0.000,1.000,16.900,2.500,0.000
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.000,2.510,1.000,N,48,238,1,14.900,1.000,0.500,15.000,0.000,1.000,34.900,2.500,0.000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.000,1.900,1.000,N,138,7,1,12.100,7.250,0.500,0.000,0.000,1.000,20.850,0.000,1.250
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.000,1.430,1.000,N,107,79,1,11.400,1.000,0.500,3.280,0.000,1.000,19.680,2.500,0.000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [9]:
df.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
count,3066766.000,2995023.000,3066766.000,2995023.000,3066766.000,3066766.000,3066766.000,3066766.000,3066766.000,3066766.000,3066766.000,3066766.000,3066766.000,3066766.000,2995023.000,2995023.000
mean,1.730,1.363,3.847,1.497,166.398,164.393,1.194,18.367,1.538,0.488,3.368,0.518,0.982,27.020,2.274,0.107
std,0.444,0.896,249.584,6.475,64.244,69.944,0.529,17.808,1.790,0.103,3.827,2.018,0.183,22.164,0.772,0.356
min,1.000,0.000,0.000,1.000,1.000,1.000,0.000,-900.000,-7.500,-0.500,-96.220,-65.000,-1.000,-751.000,-2.500,-1.250
25%,1.000,1.000,1.060,1.000,132.000,114.000,1.000,8.600,0.000,0.500,1.000,0.000,1.000,15.400,2.500,0.000
50%,2.000,1.000,1.800,1.000,162.000,162.000,1.000,12.800,1.000,0.500,2.720,0.000,1.000,20.160,2.500,0.000
75%,2.000,1.000,3.330,1.000,234.000,234.000,1.000,20.500,2.500,0.500,4.200,0.000,1.000,28.700,2.500,0.000
max,2.000,9.000,258928.150,99.000,265.000,265.000,4.000,1160.100,12.500,53.160,380.800,196.990,1.000,1169.400,2.500,1.250


In [10]:
#df.duration.describe(percentiles=[0.1,0.90,0.95,0.98,0.99,0.999])

In [11]:
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [12]:
df.duration.describe()

count   3066766.000
mean         15.669
std          42.594
min         -29.200
25%           7.117
50%          11.517
75%          18.300
max       10029.183
Name: duration, dtype: float64

In [13]:
((df.duration >= 1) & (df.duration <= 60)).mean()

0.9812202822125979

In [14]:
numerical = ['trip_distance']
categorical = ['PULocationID', 'DOLocationID']

def process_data(url):
    df = pd.read_parquet(url)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df[categorical] = df[categorical].astype(str)
    
    return df

In [15]:
train_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'
val_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet'

df_train = process_data(train_url)
df_val = process_data(val_url)

In [16]:
len(df_train), len(df_val)

(3009173, 2855951)

In [17]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

X_train = FeatureHasher().fit_transform(train_dicts)
X_val = FeatureHasher().fit_transform(val_dicts)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

### Modeling

In [18]:
def train_model(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_train)
    y_pred2 = model.predict(X_val)
    
    mse_train = mean_squared_error(y_train, y_pred, squared=False)
    mse_val = mean_squared_error(y_val, y_pred2, squared=False)
    
    return mse_train, mse_val

In [19]:
def validate_model(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    
    return mean_squared_error(y_val, y_pred, squared=False)

In [20]:
lr = LinearRegression()
train_model(lr), validate_model(lr)

((7.649134540204061, 7.811423798092246), 7.811423798092246)

In [21]:
lasso = Lasso()
train_model(lasso)

(9.938398578736242, 10.066734566474242)

In [ ]:
elastic = ElasticNet()
train_model(elastic)

In [ ]:
ridge = Ridge(alpha=1.0)
train_model(ridge)

In [ ]:
import pickle

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((lr), f_out)

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("engineer", "tobe")
    
    mlflow.log_param("train-data-url","https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
    mlflow.log_param("test-data-url", "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")
    
    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lasso = Lasso(alpha)
    
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_train)
    y_pred2 = lasso.predict(X_val)
    
    rmse_train = mean_squared_error(y_train, y_pred, squared=False)
    rmse_val = mean_squared_error(y_val, y_pred2, squared=False)
    
    mlflow.log_metric("rmse-train", rmse_train)
    mlflow.log_metric("rmse-val", rmse_val)
    
    